#Setup

### Import packages
Clone the TensorFlow Github Repository, which contains the relevant code required to run this code.


In [ ]:
#%tensorflow_version 1.x
import tensorflow as tf
!wget https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
!unzip v2.4.1.zip &> 0
!mv tensorflow-2.4.1/ tensorflow/
import sys
# We add this path so we can import the speech processing modules.
sys.path.append("/content/tensorflow/tensorflow/examples/speech_commands/")
import input_data
import models
import numpy as np
import glob
import os
import re
import shutil
from google.colab import files
!pip install ffmpeg-python &> 0
!apt-get update && apt-get -qq install xxd

--2023-11-02 21:32:48--  https://github.com/tensorflow/tensorflow/archive/v2.4.1.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tensorflow/tensorflow/zip/refs/tags/v2.4.1 [following]
--2023-11-02 21:32:49--  https://codeload.github.com/tensorflow/tensorflow/zip/refs/tags/v2.4.1
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v2.4.1.zip.1’

v2.4.1.zip.1            [ <=>                ]  66.13M  11.3MB/s    in 6.6s    

2023-11-02 21:32:56 (9.95 MB/s) - ‘v2.4.1.zip.1’ saved [69346072]

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease


### Import your collected data
First we are going to download Pete's dataset (Speech Commands dataset) to use as a base set of "other words" and "background noise" that you can build ontop of for your dataset. Doing this will make your model work a lot better, especially if you are training it with a small amount of custom data!

**Note: this may take a couple of minutes to run!**

In [ ]:
uploaded = files.upload()

Then we can convert them into correctly trimmed WAV files and then store them in the appropriate folders in the ```DATASET_DIR```.
We will use Pete's extract_loudest_section tool which you can find more documentation about here: https://github.com/petewarden/extract_loudest_section

In [ ]:
# convert the ogg files to wavs
!mkdir wavs
!find *.ogg -print0 | xargs -0 basename -s .ogg | xargs -I {} ffmpeg -i {}.ogg -ar 16000 wavs/{}.wav
!rm -r -f *.ogg

# then use pete's tool to only extract 1 second clips from them for use with the KWS pipeline
!mkdir trimmed_wavs
!git clone https://github.com/petewarden/extract_loudest_section.git
!make -C extract_loudest_section/
!/tmp/extract_loudest_section/gen/bin/extract_loudest_section 'wavs/*.wav' trimmed_wavs/
!rm -r -f /wavs

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
# Store them in the appropriate folders
data_index = {}
os.chdir('trimmed_wavs')
search_path = os.path.join('*.wav')
for wav_path in glob.glob(search_path):
    original_wav_path = wav_path
    parts = wav_path.split('_')
    if len(parts) > 2:
        wav_path = parts[0] + '_' + ''.join(parts[1:])
    matches = re.search('([^/_]+)_([^/_]+)\.wav', wav_path)
    if not matches:
        raise Exception('File name not in a recognized form:"%s"' % wav_path)
    word = matches.group(1).lower()
    instance = matches.group(2).lower()
    if not word in data_index:
      data_index[word] = {}
    if instance in data_index[word]:
        raise Exception('Audio instance already seen:"%s"' % wav_path)
    data_index[word][instance] = original_wav_path

output_dir = os.path.join('..', 'dataset')
try:
    os.mkdir(output_dir)
except:
    pass
for word in data_index:
  word_dir = os.path.join(output_dir, word)
  try:
      os.mkdir(word_dir)
      print('Created dir: ' + word_dir)
  except:
      print('Storing in existing dir: ' + word_dir)
  for instance in data_index[word]:
    wav_path = data_index[word][instance]
    output_path = os.path.join(word_dir, instance + '.wav')
    shutil.copyfile(wav_path, output_path)
os.chdir('..')
!rm -r -f trimmed_wavs

Created dir: ../dataset/only


You can download your full custom dataset as a zipfile for use later.
Download your files from the Files area by clicking on the three dots next to the zip file.

Note: this command will take a little while to run as the combination of your data and Speech Commands dataset will be relatively large!

If you would like to create a zip of just your dataset please re-run this colab (or factory reset it to wipe the memory) and skip the line that downloads Pete's dataset.

Finally if you have room in your Google Drive and would like to load and store your dataset from there in the future you can mount your Google Drive in Colab [as described in this blog post](https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166).

In [ ]:
!zip -r myKWSDataset.zip dataset

  adding: dataset/ (stored 0%)
  adding: dataset/only/ (stored 0%)
  adding: dataset/only/1698198646138.wav (deflated 16%)
  adding: dataset/only/1698198646138 (2).wav (deflated 21%)
  adding: dataset/only/1698198646137 (1).wav (deflated 34%)
  adding: dataset/only/1698198646137.wav (deflated 15%)
  adding: dataset/only/1698198646138 (1).wav (deflated 19%)


## Processing the data

Now let's go to Edge Impulse to train a model that can also be used on the microcontroller
https://www.edgeimpulse.com/

Please create acoount here!